In [1]:
from IPython.display import clear_output
from run_pipeline import run
import torch
import numpy as np
import imageio
import yaml
import time
from helper_functions import update_yaml, change_one_token


/home/novakovm/miniconda3/envs/iris_mn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/novakovm/iris/MILOS/models.py:509: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(False, f'Unknown config parameter name = {param_name}.')
/home/novakovm/iris/MILOS/models.py:628: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(False, f'Unknown config parameter name = {param_name}.')
/home/novakovm/iris/MILOS/models.py:978: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(False, f"There are no optimizers called {self.optimizer_settings['optimization_algorithm']}.")
/home/novakovm/iris/MILOS/models.py:1433: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(self.loaders['test'].batch_size == 1, f"Mi

In [2]:

config_folder_names = { "toy_dataset" : "/home/novakovm/iris/MILOS/",
                        "crafter_dataset" : "/home/novakovm/iris/MILOS/"}

config_file_names = {   "toy_dataset" : "toy_shapes_config",
                        "crafter_dataset" : "crafter_config"}

config_full_file_paths = {}
for dataset_name in config_file_names:
    config_full_file_paths[dataset_name] = config_folder_names[dataset_name] + \
                                           config_file_names[dataset_name] + ".yaml"


K_BIT_MIN, K_BIT_MAX = 3,8
D_array =np.array([64, 32, 16, 8])
M_array = np.array([7, 3, 1]) #M_array = -np.sort(-M_array)
K_array = 2** np.arange(K_BIT_MIN, K_BIT_MAX+1)
K_array = -np.sort(-K_array)

trainers = {}
custom_logger= "/home/novakovm/iris/MILOS/CUSTOM_LOGGER.txt"

# init values for run_ids
run_id = { "toy_dataset" : 101,
        "crafter_dataset" : 501}

"""
Delimiter in log_all.txt-like files for this simulation
##################################################################################################################################
##################################################################################################################################
##################################################################################################################################
##################################################################################################################################
"""


# config setup
for dataset_name in config_file_names:
    update_yaml(yaml_folder_name = config_folder_names[dataset_name],
                yaml_file_name = config_file_names[dataset_name],
                key = None, 
                value = None)
    
    
trainers = {}

# toy dataset
dataset_name, dataset_type = "toy_dataset", "test"
trainers[dataset_name] = run(config_full_file_paths[dataset_name])

image_id_in_dataset = trainers[dataset_name].worst_imgs_ids[0]
image_index_in_dataset = np.where(trainers[dataset_name].loaders[dataset_type].dataset.image_ids == image_id_in_dataset)[0][0]

change_one_token(trainers, dataset_name, dataset_type, image_index_in_dataset)


# crafter
dataset_name, dataset_type = "crafter_dataset", "test"
trainers[dataset_name] = run(config_full_file_paths[dataset_name])

image_id_in_dataset = trainers[dataset_name].worst_imgs_ids[0]
image_index_in_dataset = np.where(trainers[dataset_name].loaders[dataset_type].dataset.image_ids == image_id_in_dataset)[0][0]

change_one_token(trainers, dataset_name, dataset_type, image_index_in_dataset = None)

Total Trainable Params in thousands: 1731

Total Memory =  25.43 GB.
Reserved Memory =  0.00 GB.
Allocated inside Reserved Memory =  0.00 GB.
Free inside Reserved Memory =  0.00 GB.

Global Free memory on the cuda:0 =  24.52 GB.
Total GPU Memory occupied on the cuda:0 =  25.43 GB.

Inverse of training data variance term is equal to =  7.9
Testing Started
Average test total loss =  34464 e-6
Average test total reconstruction loss =  23912 e-6 (69 %)
Average test total commitment loss =  2110 e-6 (6 %)
Average test total VQ loss =  8442 e-6 (24 %)
Testing Ended


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (3000, 800) to (3008, 800) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (3000, 800) to (3008, 800) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (3000, 800) to (3008, 800) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: inp

KeyboardInterrupt: 

In [ ]:
# image_id_in_dataset = trainers[dataset_name].worst_imgs_ids[0]
# image_index_in_dataset = np.where(trainers[dataset_name].loaders[dataset_type].dataset.image_ids == image_id_in_dataset)[0]
# image_index_in_dataset

In [ ]:
# loader = trainers[dataset_name].loaders[dataset_type]
# loader_dataset = loader.dataset
# #loader_dataset
# loader_dataset.image_ids[image_index_in_dataset[0]]

In [ ]:
# dataset_name, dataset_type = "toy_dataset", "test"
# image_id_in_dataset = trainers[dataset_name].worst_imgs_ids[0]
# image_index_in_dataset = np.where(trainers[dataset_name].loaders[dataset_type].dataset.image_ids == image_id_in_dataset)[0][0]
# image_index_in_dataset

In [ ]:
# loader = trainers[dataset_name].loaders[dataset_type]
# loader_dataset = loader.dataset
# image_batch,image_id_batch = loader_dataset[image_index_in_dataset]

In [ ]:
# image_batch.shape#,image_id_batch

In [ ]:
print("Finish")